In [1]:
from math import floor, log2
from sympy import prime
import random

In [72]:
#!/usr/bin/python

from binascii import hexlify
import math
import os
import sys
import gmpy2
from gmpy2 import mpz, random_state, next_prime, mpz_urandomb, is_prime

_DEBUG = False

FLAG  = mpz(hexlify(b"flag{yay}"), 16)
SEED  = mpz(hexlify(b"weee").decode(), 16)
STATE = random_state(SEED)
def get_prime(state, bits):
    return next_prime(mpz_urandomb(state, bits) | (1 << (bits - 1)))

def get_smooth_prime(state, bits, smoothness=16):
    p = mpz(2)
    p_factors = [p]
    while p.bit_length() < bits - 2 * smoothness:
        factor = get_prime(state, smoothness)
        p_factors.append(factor)
        p *= factor

    bitcnt = (bits - p.bit_length()) // 2

    while True:
        prime1 = get_prime(state, bitcnt)
        prime2 = get_prime(state, bitcnt)
        tmpp = p * prime1 * prime2
        if tmpp.bit_length() < bits:
            bitcnt += 1
            continue
        if tmpp.bit_length() > bits:
            bitcnt -= 1
            continue
        if is_prime(tmpp + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = tmpp + 1
            break

    p_factors.sort()

    return (p, p_factors)

In [73]:
e = 0x10001

while True:
    p, p_factors = get_smooth_prime(STATE, 1024, 16) # 16 smooth
    if len(p_factors) != len(set(p_factors)):
        continue
    # Smoothness should be different or some might encounter issues.
    q, q_factors = get_smooth_prime(STATE, 1024, 17) # 17 smooth
    if len(q_factors) != len(set(q_factors)):
        continue
    factors = p_factors + q_factors
    if e not in factors:
        break

n = p * q

In [85]:
(n-1)%6

mpz(0)

In [78]:
from sympy import isprime
isprime(p)

True

In [74]:
p, p_factors

(mpz(152943054057234143564403676742432386870006054766143977453858870302238068545918534264650151819747090956029677845596661800601342110624294235406969834751707419017752916279373543236245723177401994103244660031403682289060889908841301373580996705353830539087251725604027548933594888692538154959689080659570782209967),
 [mpz(2),
  mpz(9091),
  mpz(12739),
  mpz(33029),
  mpz(33521),
  mpz(33911),
  mpz(34123),
  mpz(34259),
  mpz(34421),
  mpz(35023),
  mpz(35671),
  mpz(36251),
  mpz(36467),
  mpz(36551),
  mpz(37097),
  mpz(37409),
  mpz(38593),
  mpz(39019),
  mpz(39607),
  mpz(41491),
  mpz(42557),
  mpz(42923),
  mpz(44189),
  mpz(44741),
  mpz(45953),
  mpz(46171),
  mpz(46399),
  mpz(46523),
  mpz(46687),
  mpz(46867),
  mpz(47087),
  mpz(47123),
  mpz(47969),
  mpz(48611),
  mpz(48647),
  mpz(49853),
  mpz(49993),
  mpz(52361),
  mpz(53129),
  mpz(53849),
  mpz(54139),
  mpz(54193),
  mpz(54409),
  mpz(54667),
  mpz(55609),
  mpz(56369),
  mpz(56431),
  mpz(56591),
  mpz(56611),
  

In [58]:
n%63901

mpz(14970)

In [51]:
p%9091

mpz(1)

In [ ]:




m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)

c = pow(FLAG, e, n)

print(f'n = {n.digits(16)}')
print(f'c = {c.digits(16)}')


In [2]:

def lcm(a,b):
    return abs(a*b)//extended_gcd(a,b)[0]
ascii32bit = "abcdefghijklmnopqrstuvwxyz.,!?' "
def ascii32bit_encode(s: str):
    acc = 0
    i = 0
    for c in s:
        acc+=(ascii32bit.index(c) if c in ascii32bit else 31)<<i
        i+=5
    return acc
# edge case where last char has leading zeros, need to adjust bit shift offset accordingly
def ascii32bit_decode(s: int):
    acc = s
    ans = ""
    n = round(log2(s)) - 5
    n = n+(5-n%5) if n%5!=0 else n
    for i in range(n, -1, -5):
        temp = acc >> i
        temp2 = temp << i
        ans=ascii32bit[temp]+ans
        acc -= temp2
    return ans
def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m
# ported from https://stackoverflow.com/questions/2921406/calculate-primes-p-and-q-from-private-exponent-d-public-exponent-e-and-the
# given d e and n, find p and q
def pq_from_den(d, e, n):
    k = d*e-1
    if (k%2==0):
        r = k
        t = 0
        while (r%2==0):
            r = r >> 1
            t+=1
        success = False
        y = -1
        for i in range(1,101):
            g = -1
            g = random.randrange(0,n)
            y = fast_mod_exp(g,r,n)
            if y==1 or y==n-1:
                continue
            for j in range(1, t):
                x = fast_mod_exp(y,2,n)
                if x==1:
                    success = True
                    break
                if x==n-1:
                    continue
                y = x
            x = fast_mod_exp(y, 2, n)
            if (x==1):
                success=True
                break
        if (success):
            p = gcd(y-1, n)[0]
            q = n//p
            return p, q
        else:
            assert False, "not found"
    else:
        assert False, "not found"
def extended_gcd(a, b):
    r1 = b
    r2 = a
    u1 = 0
    v1 = 1
    u2 = 1
    v2 = 0
    while r2 != 0:
        q = r1//r2
        r3 = r1
        u3 = u1
        v3 = v1
        r1 = r2
        u1 = u2
        v1 = v2
        r2 = r3 - q * r2
        u2 = u3 - q * u2
        v2 = v3 - q * v2
    return (r1, u1, v1)

In [52]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from sympy import root
n = 0xb03ea698ce2b51fb00e11e6fbaf1e5373dc5b0c70eb2b14a36d21e8667be8774eee51f6050a10237f6b24f21204fc8013681e7ed72ed051188f3274aae8f1de0d39389b514c196fa82c98a270bfabefd044da8c687b0e114ebbde82536c0709ac5ad81bfe0077e9d9b798ad5abecee52767e68f8060c45936521fd93893102eb1676f2ff41324a7a6b3dff2e830538e06d25934e9f14bf6b40ab5674fe648e314bf06f84282f5ef52bc1401de3a42eb66e64bcdadd2674348e5bdb7016feda44d719af387a948ad81cbaed10213dd930fc7bc7677d8c4cdab0645d0ff15e6ad6ca37135942c3be08f23e7be0992c8b3370dcdc31045e086d823107fb2e443dc9
c = 0xa913a96e215b5aa79c702d27fa375c73d06787639c4131fb32877cafefaa8faf70e15f6a17ef2a9a6f5310b157cb287b740e77cb5385081d1853a9104bc16357b259fa2d146bd87398d4ef6f1c078289812952c67792cf9cd745049aeb9d4ab4dff2825a9c0b3381f19b2a67164f9d4de33c25f98bc2f224feb5507b531e1a1c7be5ed2d8ddd01f3fae37245e8cf99c75a21848993d445e1d6d69d555a3e6cc8055704fdde88df9084bda3ea65a9384fa64bf8df4d88946449526320c15d4d2d871638070489adf3f8c95caffeab40b0d137a9319be20cdf6ebbaf037f62093d9bd33edd4ffd7e1929b9ab06252956fd85250a0515ef2b4e035017be5702cdd3
e = 0x10001

In [71]:
from sympy import factorint, gcd, prime
from math import log, floor
B = 100000
M = 1
i = 1
while (prime(i)<=B):
    M*=pow(prime(i), floor(log(B, prime(i))))
    i+=1
    print(i, end="")
a = 2
g = gcd(pow(a, M, n)-1, n)
g

23456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169

1